In [4]:
from coffea.util import load
import os
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt

local=""#"/run/user/1000/gvfs/sftp:host=t3ui02.psi.ch,user=mmalucch/"
prefix = local+str(
    os.environ.get("HH", "/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/out_HH4b/")
)

input = prefix + "/out_parton_matching_pnetreg_btaglead_nokinjet/output_all.coffea"
# input = prefix + "out_parton_matching_pnetreg_btaglead/output_all.coffea"
o = load(f"{input}")
col=o["columns"]['GluGlutoHHto4B']['GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_kl-1p00_kt-1p00_c2-0p00_2022']['baseline']
for k in col.keys():
    col[k]=col[k].value[col[k].value!=-999]

In [15]:
unflat_pt = ak.unflatten(col["JetGoodBTagOrder_ptPnetRegNeutrino"], col["JetGoodBTagOrder_N"])
unflat_btag = ak.unflatten(col["JetGoodBTagOrder_btagPNetB"], col["JetGoodBTagOrder_N"])
unflat_pt_order = ak.unflatten(col["JetGoodPtOrder_ptPnetRegNeutrino"], col["JetGoodPtOrder_N"])

print(unflat_pt)
# print(unflat_btag)
print(unflat_pt_order)


def find_idx(i):
    jet_pt = unflat_pt[:, i]

    jet_pt_order = unflat_pt_order

    # for each elemtnt in the first dimension of jet_pt_order, check if the jet_pt is in the jet_pt_order
    # if yes, return the index of the jet_pt in the jet_pt_order
    # if not, return -1
    # index = np.array([np.where(jet_pt_order[:,i] == jet_pt) for i in range(3)])
    index=np.empty((len(jet_pt_order)), dtype=int)
    for j in range(len(jet_pt_order)):
        index[j]=(np.where(jet_pt_order[j] == jet_pt[j]))[0][0]


    print(index)
    return index


indexes=np.ndarray((4, len(unflat_pt)), dtype=int)
for k in range(4):
    indexes[k]=find_idx(k)

print(indexes)


[[111, 59.4, 47.7, 45.5], [164, 146, 44.9, ... 125, 46.1], [166, 67.1, 44.1, 77.8]]
[[111, 59.4, 47.7, 45.5], [164, 146, 54.1, ... 60.6, 46.1], [166, 77.8, 67.1, 44.1]]
[0 0 0 ... 2 1 0]
[1 1 1 ... 0 2 2]
[2 3 2 ... 3 0 3]
[3 2 3 ... 1 3 1]
[[0 0 0 ... 2 1 0]
 [1 1 1 ... 0 2 2]
 [2 3 2 ... 3 0 3]
 [3 2 3 ... 1 3 1]]


In [17]:
pt_cuts=[80, 60, 45, 40]
# pt_cuts=[40,40,40, 40]
mean=(unflat_btag[:, 0]+unflat_btag[:, 1])/2
mask_mean = np.array(mean>0.65)
print(mean[mean>0.65])
print(np.sum(mean>0.65))
print(mask_mean)

def eff(i):
    jet_pt = unflat_pt[:, i]
    # print(jet_pt)
    jet_btag = unflat_btag[:, i]
    # print(jet_btag)
    total = len(jet_pt)
    # print(total)
    index = indexes[i]
    print(index)
    # check if the jet_pt is above the pt cut
    # create array of pt_cuts for each event in the first dimension of jet_pt defined by the index
    pt_cuts_array = np.array([pt_cuts[j] for j in index])
    print(pt_cuts_array)

    mask = jet_pt > pt_cuts_array
    print(mask)
    jet_masked = jet_btag[mask]
    # print(jet_masked)
    # print(jet_pt[mask])
    # compute the number of jets above the pt cut
    above = len(jet_masked)
    # print(above)
    eff = above/total
    print(eff)

    return mask


masks=np.ndarray((4, len(unflat_pt)), dtype=object)
for k in range(4):
    masks[k]=eff(k)

mask_total = np.logical_and.reduce(masks)
print(mask_total)
print(len(mask_total))
# mask_total = mask_total[mask_mean] #??
print(len(mask_total))
print(mask_total.sum())
print(mask_total.sum()/len(mask_total))


[0.994, 0.996, 0.97, 0.998, 0.999, 0.999, ... 0.665, 0.994, 0.99, 0.976, 0.954]
27721
[ True  True  True ...  True  True  True]
[0 0 0 ... 2 1 0]
[80 80 80 ... 45 60 80]
[True, True, True, True, True, True, True, ... True, True, True, True, True, True]
0.9897751919303198
[1 1 1 ... 0 2 2]
[60 60 60 ... 80 45 45]
[False, True, True, True, True, True, True, ... True, True, True, True, True, True]
0.9883636864392192
[2 3 2 ... 3 0 3]
[45 40 45 ... 40 80 40]
[True, True, True, True, True, True, True, ... True, True, True, True, True, True]
0.9885358212552071
[3 2 3 ... 1 3 1]
[40 45 40 ... 60 40 60]
[True, True, True, True, True, True, True, ... True, True, True, True, True, True]
0.9861603607945744
[False True True ... True True True]
29047
29047
27874
0.9596171721692429


In [7]:
a=np.array([[1,2,3,4,5,6,7,8,9,10]])
b=np.array([2,3,8,9,10, 11, 12, 13, 14, 1])
np.where(a[:]==b[:])

(array([], dtype=int64), array([], dtype=int64))